In [1]:
import os
from typing import Any, Dict, List
import sys

from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import Tongyi
from langchain.utilities import BingSearchAPIWrapper
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline, TextStreamer
from langchain_openai import ChatOpenAI
from langchain_core.outputs import LLMResult
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import BaseMessage
import torch

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CHAT_MODEL_PATH = "/mnt/nfs/zsd_server/models/huggingface/llama-3-chinese-8b-instruct-v3/"

In [3]:
os.environ["BING_SUBSCRIPTION_KEY"] = ""
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

# model_name="llama-3-chinese-8b-instruct-v3-f16"
model_name="qwen2:7b"

In [4]:
search = BingSearchAPIWrapper()

res = search.run("北京的面积有多大？")
res

'<b>北京</b>市地处华北平原西北隅，地势西北高、东南低。地形<b>的</b>特点是山地<b>多</b>、平原少，北部为中低山区，东南部是缓斜<b>的</b>平原，其中山区<b>面积</b>约占61%，平原<b>面积</b>约占39%。全市平均海拔43.5米。<b>北京</b>市平原<b>的</b>海拔高度在20～60米，山地一般海拔1000～1500米。 北京的面积虽然只有1.6万平方公里， 但是却完全可以傲视群雄！ 北京＝2.5×上海. （上海面积6340平方公里） . 北京=9.89×厦门. （厦门面积1699.39平方公里） . 北京=1.26×大连. （大连面积1.3237万平方公里） . 北京=1.4×天津. （天津面积1.1946万平方公里） . 北京=8.42×深圳. （深圳面积1996.85平方公里） . 北京=1.48×青岛. （青岛面积11282平方公里） . 北京=15.2×香港. （香港面积1104.43平方公里） 东京首都圈<b>的面积</b>大小和整个<b>北京</b>市相差并不远，甚至没<b>有北京大</b>。 而人口，却比<b>北京</b>更<b>多</b>。 另外，机构预计到2035年，全世界最富有<b>的</b>十大城市中，东京圈排名第二（现在也是第二），总GDP比上海、<b>北京</b>仍然高出不少。 北京市平原区面积为<b>6200平方千米</b>，<b>约</b>占总面积的38%，山区面积10200平方千米，约占总面积的62%，北京的地形西北高，东南低。 北京市平均海拔43.5米。 北京平原的海拔高度在20～60米，山地一般海拔1000～1500米。 北京位于北纬39度56<b>分</b>、东经116度20<b>分</b>，地处华北大平原的北部，东面与天津市毗连，其余均与河北省相邻。 北京，简称“京”，是中华人民共和国的首都，是全国的政治中心、文化中心，是世界著名古都和现代化国际城市。 <b>北京</b>，简称“京”，是中华人民共和国<b>的</b>首都、直辖市、国家中心城市、超<b>大</b>城市、国际<b>大</b>都市，全国政治中心、文化中心、国际交往中心、科技创新中心，是中国共产党中央委员会、中华人民共和国中央人民政府、全国人民代表大会、中国人民政治协商会议全国 对于“北京有多大？. ”这类问题，当然可以理解为

In [5]:
res = search.run("纽约的面积有多大？")
res

'<b>纽约</b>（英语：New York），隶属于美国 <b>纽约</b>州 ，美国第一<b>大</b>城市，位于美国东北部沿海 哈德逊河 口，濒临 大西洋 ，属 温带大陆性湿润气候 。. 总<b>面积</b>1214平方千米，下辖5个区，市政厅驻曼哈顿市政厅公园内。. [22]截至2022年，总人口约834万人。. [43] <b>纽约</b>原为 ... 纽约市整体面积约<b>1214平方公里</b>（<b>468.9平方</b>英里），其中425平方公里（164.1平方英里）为水域，789平方公里（304.8平方英里）为陆地。. 纽约市地处美国东北纽约州东南哈德森河口，大约在华盛顿特区和波士顿中间位置，紧邻的哈德逊河让纽约市享有运航之 ... <b>阿帕拉契高原区是</b>纽约州面积最大的地理区域，几乎涵盖了该州南方的大部分地区。纽约州最主要的河流有 哈德逊河、莫华克河、圣罗伦斯河、以及尼加拉河。 [1] [4] 作为世界经济和国际金融<b>的</b>神经中枢之一，<b>纽约</b>占据了区域内<b>的</b>核心地位，而位于波士顿郊区<b>的</b>128号公路两侧则聚集了数以千计<b>的</b>研究机构和高科技企业，被称为&quot;美国东海岸<b>的</b>硅谷&quot;。 <b>纽约面积多大纽约</b>是世界特大城市之一，美国最大<b>的</b>金融、商业、贸易和文化中心。 位于美国东北部哈得孙河注入大西洋<b>的</b>河口处。 市区<b>面积</b>945平方公里，其中水面168平方公里。 <b>纽约多大</b>，最长多少？<b>面积</b>多少？<b>纽约</b>（New York）是美国第一<b>大</b>都市和第一<b>大</b>商港，它不仅是美国<b>的</b>金融中心，也是全世界金融中心之一。<b>纽约</b>位于<b>纽约</b>州东南哈得孙河口，濒临大西洋。它由五个区组成：曼哈顿、布鲁克林、 截至2017年，<b>纽约</b>市人口约851万人，居住在789平方公里<b>的</b>土地上。而<b>纽约大</b>都市圈则<b>有</b>2000万人左右，仅次于东京、墨西哥城、孟买，位居世界第四位。2017年，<b>纽约</b>地区生产总值已达到9007亿美元。扩展资料： <b>纽约的的</b>地理环境： 1、地形地貌 根据最新的统

In [6]:
# chat_model = AutoModelForCausalLM.from_pretrained(
#     CHAT_MODEL_PATH,
#     # load_in_8bit=True,
#     device_map='auto',
#     torch_dtype=torch.bfloat16,
# )
# tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_PATH)
# streamer = TextStreamer(tokenizer)
# pipe = pipeline(
#     "text-generation",
#     model=chat_model,
#     tokenizer=tokenizer,
#     max_length=4096,
#     truncation=True,
#     repetition_penalty=1.2,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.95,
#     streamer=streamer,
#     return_full_text=False,
# )
# local_llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
class MyStreamingStdOutCallbackHandler(StreamingStdOutCallbackHandler):
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        sys.stdout.write("\n\n")
        sys.stdout.flush()

    # def on_chat_model_start(
    #     self,
    #     serialized: Dict[str, Any],
    #     messages: List[List[BaseMessage]],
    #     **kwargs: Any,
    # ) -> None:
    #     sys.stdout.write(f"{messages}\n\n")
    #     sys.stdout.flush()

callbacks = [MyStreamingStdOutCallbackHandler()]
local_llm = ChatOpenAI(
    model_name=model_name,
    openai_api_key="your-api-key",
    openai_api_base="http://localhost:11434/v1/",
    temperature=0.6,
    streaming=bool(callbacks),
#     callbacks=callbacks,
    stop=['<|eot_id|>', 'Observation:', 'Observation:\n'],
)

# langchain自带的bing_search，它的key模型总会输出成`Bing Search`，造成无法匹配
# 而自定义的tool放在load_tools里会报错，官方给的方案竟然是在最后追加上去

In [8]:
# tools = load_tools(["bing-search", "llm-math"], llm=local_llm)
search_tool = Tool(
        name="Search",
        func=search.run,
        description="A wrapper around Search. Useful for when you need to answer questions about current events. Input should be a search query.",
    )
tools = load_tools(["llm-math"], llm=local_llm)
tools = tools + [search_tool]

In [9]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Comp

In [10]:
agent = initialize_agent(
    tools,
    local_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    stop=['Observation:', 'Observation:\n'],
)

# agent.run("北京的面积和纽约的面积差是多少？")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


# llama3模型不用自己的模板就无法停止
# 中文问题容易使模型回答时使用中文标点，造成langchain无法匹配`Action: `

In [11]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\nSearch(query: str) -> str - A wrapper around Search. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator, Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original i

In [12]:
# PREFIX = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>Answer the following questions as best you can. You have access to the following tools:"""
# FORMAT_INSTRUCTIONS = """If you can give the final answer:
# Final Answer: the final answer to the original input question.
# After the final answer, you can <|eot_id|><|eot_id|>

# If you can not give the final answer:
# Thought: you should always think about what to do
# Action: the action to take, should be one of [Calculator, Search]
# Action Input: the input to the action.
# After Action Input, you can <|eot_id|><|eot_id|>"""
# SUFFIX = """<|start_header_id|>user<|end_header_id|>: Begin!

# Question: {input}<|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>:Thought: {agent_scratchpad}"""

# agent = initialize_agent(
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     tools=tools,
#     llm=local_llm,
#     agent_kwargs={
#         'prefix':PREFIX,
#         'format_instructions':FORMAT_INSTRUCTIONS,
#         'suffix':SUFFIX
#     },
#     verbose=True,
#     handle_parsing_errors=True,
#     stop=['<|eot_id|>', 'Observation:', 'Observation:\n'],
# )

In [13]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\nSearch(query: str) -> str - A wrapper around Search. Useful for when you need to answer questions about current events. Input should be a search query.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator, Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original i

In [14]:
# agent.run("How many people are there in the district where Jinling Middle School is located?")
agent.run("金陵中学所在的行政区有多少人?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
这个问题涉及到具体的数据，需要搜索以获取最新的信息。

Action: Search
Action Input: "南京市玄武区人口数量
Observation: 根据南京市玄武区第七次全国人口普查结果，现将2020年11月1日零时我区常住人口[2]的基本情况公布如下： 一、常住人口. 全区常住人口为537825人，与2010年我区第六次全国人口普查的651957人相比，十年共减少114132人，下降17.51%，年平均下降率为1.91%。 二、户别人口. 全区共有家庭户[3]173057户，集体户35970户，家庭户人口为417281人，集体户人口为120544人。 平均每个家庭户的人口为2.41人,比2010年我区第六次全国人口普查的2.64人减少0.23人。 三、性别构成. 全区常住人口中，男性人口为268349人，占49.90%， 女性人口为269476人，占50.10%。 南京市第七次全国人口普查领导小组办公室. 2021年5月24日. 根据南京市第七次全国人口普查结果，现将2020年11月1日零时我市常住人口 [2]的基本情况公布如下： 一、常住人口. 全市常住人口为9314685人，与2010年我市第六次全国人口普查的8003744人相比，十年共增加1310941人，增长16.38%，年平均增长率为1.53%。 二、户别人口. 全市共有家庭户 [3]3138775户，集体户461313户，家庭户人口为7795032人，集体户人口为1519653人。 平均每个家庭户的人口为2.48人，比2010年我市第六次全国人口普查的2.77人减少0.29人。 三、地区人口. 主办单位：<b>南京市</b>统计局 支持单位：<b>南京市</b>城市数字治理中心 苏ICP备05004952号-7 网站标识码：3201000016 苏公网安备 32010502010154号 您是本站第 显示网站总访问量 位访客 <b>玄武区</b>，江苏省<b>南京市</b>辖区，位于<b>南京市</b>中部，是国家重要的科研、文化、商务中心，华东地区现代服务业基地，是<b>南京</b>市委、<b>市</b>政府及众多省、<b>市</b>直机关所在地，也是中国人民解放军东部战区、中国人民解放军东部战区空军等部

'南京市玄武区的常住人口为537,825人。南京市的常住人口为9,314,685人。请注意，这些数据可能有所变化，请参考最新的统计信息以获得准确的人口数量。'